<a href="https://colab.research.google.com/github/j143/in-car-voice-assistant/blob/colab/notebooks/edge_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/j143/in-car-voice-assistant

Cloning into 'in-car-voice-assistant'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 244 (delta 107), reused 216 (delta 88), pack-reused 0 (from 0)
Receiving objects: 100% (244/244), 178.03 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [3]:
%cd in-car-voice-assistant/
#!git stash save
!git pull https://github.com/j143/in-car-voice-assistant data

/content/in-car-voice-assistant
From https://github.com/j143/in-car-voice-assistant
 * branch            data       -> FETCH_HEAD
Updating 359449b..f0f2eb2
Fast-forward
 data/automotive_domain_train_1k.jsonl | 1000 +++++++++++++++++++++++++++++++++
 1 file changed, 1000 insertions(+)
 create mode 100644 data/automotive_domain_train_1k.jsonl


In [2]:
%cd in-car-voice-assistant

/content/in-car-voice-assistant


In [4]:
%ls

config/            pipeline/                            requirements-train.txt
data/              QUICKSTART_EXPERIMENTS.md            requirements.txt
DEPLOYMENT.md      README.md                            role.md
Dockerfile.edge    requirements-awq.txt                 scripts/
Dockerfile.orin    requirements-min.txt                 STRATEGIC_ALIGNMENT.md
IMPLEMENTATION.md  requirements-nlu.txt                 tests/
models/            requirements-rag.txt
notebooks/         requirements-train-colab-freeze.txt


In [4]:
import json
import subprocess
from pathlib import Path

## 1. Baseline Evaluation

Rule-based classifier on automotive test set.

In [3]:
import sys
import json
import subprocess

# Run baseline evaluation
result = subprocess.run(
    [sys.executable, "scripts/evaluate_domain.py", "--test", "data/automotive_domain_test.jsonl"],
    capture_output=True,
    text=True,
    env={"PYTHONPATH": "."},
)

print("--- Subprocess Standard Output ---")
print(result.stdout)
print("--- End Subprocess Standard Output ---")
print("--- Subprocess Standard Error ---")
print(result.stderr)
print("--- End Subprocess Standard Error ---")

# Attempt to parse the JSON output.
try:
    # Find the start and end of the JSON object in the stdout
    json_start_index = result.stdout.find('{')
    json_end_index = result.stdout.rfind('}')

    if json_start_index != -1 and json_end_index != -1 and json_end_index > json_start_index:
        json_string = result.stdout[json_start_index : json_end_index + 1]
        baseline = json.loads(json_string)
        print(json.dumps(baseline, indent=2))
    else:
        print("Warning: Could not find a complete JSON object in the subprocess output.")
        print("Please inspect the full output above.")

except json.JSONDecodeError as e:
    print(f"JSONDecodeError when parsing: {e}")
    print("Could not parse the extracted JSON string. Please inspect the full output above.")
except IndexError:
    print("Error: Subprocess stdout was empty or did not contain a newline before JSON parsing attempt.")


--- Subprocess Standard Output ---
{
  "total_samples": 15,
  "command_accuracy": 0.2,
  "technical_term_recognition_rate": 0.1667,
  "error_code_recognition_rate": 0.0,
  "avg_latency_ms": 58.67,
  "p95_latency_ms": 185.42,
  "latency_target_met": true,
  "accuracy_target_met": false
}

=== Performance Summary ===
Command Accuracy: 20.0% (Target: ≥90%)
Avg Latency: 58.67ms (Target: <100ms)
Technical Term Recognition: 16.7%
Error Code Recognition: 0.0%

✓ Accuracy Target Met: False
✓ Latency Target Met: True

--- End Subprocess Standard Output ---
--- Subprocess Standard Error ---
2025-12-18 08:54:41.160487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766048081.229874     802 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766048081.2403

## 2. QLoRA Fine-Tuning

Train LoRA adapters on automotive domain data.

In [5]:
!pip install -r requirements-train.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.7 MB/s eta 0:00:00


In [5]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.7/774.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.0 MB/s eta 0:00:00


In [16]:
# Train QLoRA (requires GPU + requirements-train.txt)
!python scripts/train_qlora.py --model microsoft/phi-2 --train data/automotive_domain_train_1k.jsonl --output checkpoints/automotive-adapter --epochs 3

2025-12-21 08:25:39.744379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766305539.777963   21133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766305539.787952   21133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766305539.808908   21133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766305539.808936   21133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766305539.808943   21133 computation_placer.cc:177] computation placer alr

In [11]:
import json
from pathlib import Path
import jsonschema

# 1. Install jsonschema if not already installed
!pip install jsonschema

# Define the expected schema based on the example provided by the user
json_schema = {
    "type": "object",
    "properties": {
        "id": {"type": "integer"},
        "intent": {"type": "string"},
        "text": {"type": "string"},
        "error_codes": {"type": "array", "items": {"type": "string"}},
        "subsystem": {"type": "string"},
        "technical_terms": {"type": "array", "items": {"type": "string"}},
        "command": {"type": "string"},
        "severity": {"type": "string"},
        "expected": {"type": "string"},
        "source": {"type": "string"},
        "variant_type": {"type": "string"}
    },
    "required": [
        "id",
        "intent",
        "text",
        "error_codes",
        "subsystem",
        "technical_terms",
        "command",
        "severity",
        "expected"
    ],
    "additionalProperties": False
}

file_path = Path("data/automotive_domain_train_1k.jsonl")

if not file_path.exists():
    print(f"Error: File not found at {file_path}")
else:
    print(f"Validating and cleaning schema for {file_path}...")
    errors_found_during_parsing = 0
    errors_found_during_validation = 0
    successfully_parsed_objects = []

    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            line_num = i + 1
            original_line = line.strip()
            if not original_line:  # Skip empty lines
                continue

            try:
                # Use JSONDecoder.raw_decode to extract only the first valid JSON object
                decoder = json.JSONDecoder()
                data, end_idx = decoder.raw_decode(original_line)

                # Check if there's indeed extra data after the first JSON object
                if end_idx < len(original_line):
                    print(f"Warning: Line {line_num} contained 'Extra data' beyond the first JSON object. Only the first object was processed.")
                    # Optionally print the full problematic line for debugging:
                    # print(f"  Full Line Content: {original_line}")
                    # print(f"  Extra data found from char {end_idx}: '{original_line[end_idx:]}'")
                    errors_found_during_parsing += 1

                # Now validate the extracted data against the schema
                jsonschema.validate(instance=data, schema=json_schema)
                successfully_parsed_objects.append(data)

            except json.JSONDecodeError as e:
                print(f"JSON parsing error on line {line_num}: {e}")
                print(f"  Problematic line content (snippet): {original_line[:200]}...")
                errors_found_during_parsing += 1
            except jsonschema.ValidationError as e:
                print(f"Schema validation error on line {line_num}: {e.message}")
                print(f"  Problematic data: {data}")
                errors_found_during_validation += 1
            except Exception as e:
                print(f"An unexpected error occurred on line {line_num}: {e}")
                print(f"  Problematic line content (snippet): {original_line[:200]}...")
                errors_found_during_parsing += 1 # Count as parsing error for simplicity

    total_errors = errors_found_during_parsing + errors_found_during_validation
    if total_errors == 0:
        print("\nAll lines in the JSONL file are valid and conform to the schema (one JSON object per line). No cleaning needed.")
    else:
        print(f"\nCompleted validation with {total_errors} errors ({errors_found_during_parsing} parsing errors, {errors_found_during_validation} validation errors).")
        print(f"Successfully processed {len(successfully_parsed_objects)} valid JSON objects.")

        # Create a new, cleaned file with only the successfully parsed objects
        cleaned_file_path = file_path.with_name(file_path.stem + '_cleaned.jsonl')
        print(f"Rewriting a cleaned version of the file to {cleaned_file_path}...")
        with open(cleaned_file_path, 'w') as out_f:
            for obj in successfully_parsed_objects:
                out_f.write(json.dumps(obj) + '\n')
        print(f"Cleaned file saved to {cleaned_file_path}. Please use this file for training (e.g., --train {cleaned_file_path}).")


Validating schema for data/automotive_domain_train_1k.jsonl...

All lines in the JSONL file are valid and conform to the schema.


In [2]:
%cd in-car-voice-assistant

/content/in-car-voice-assistant


In [7]:
!python scripts/train_qlora.py --model microsoft/phi-2 --epochs 15 --output checkpoints/automotive-adapter

2025-12-18 10:08:41.977984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766052521.998403   20083 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766052522.004666   20083 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766052522.020793   20083 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766052522.020828   20083 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766052522.020832   20083 computation_placer.cc:177] computation placer alr

In [25]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('microsoft/phi-2', trust_remote_code=True)
for name, mod in list(model.named_modules())[:120]:
    if mod.__class__.__name__.lower().find('linear')>=0:
        print(name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.dense
model.layers.0.mlp.fc1
model.layers.0.mlp.fc2
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.dense
model.layers.1.mlp.fc1
model.layers.1.mlp.fc2
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.dense
model.layers.2.mlp.fc1
model.layers.2.mlp.fc2
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.dense
model.layers.3.mlp.fc1
model.layers.3.mlp.fc2
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.dense
model.layers.4.mlp.fc1
model.layers.4.mlp.fc2
model.layers.5.self_attn.q_proj
model.layers.5.self_attn.k_proj
model.layers.5.self_attn.v_proj
model.layers.5.self_attn.dense
model.la

In [ ]:
!python scripts/evaluate_intent.py data/automotive_domain_test.jsonl --adapter checkpoints/automotive-adapter

2025-12-21 08:54:23.175487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766307263.195589   28484 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766307263.201645   28484 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766307263.217502   28484 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766307263.217530   28484 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766307263.217533   28484 computation_placer.cc:177] computation placer alr

In [19]:
!python scripts/evaluate_intent.py data/automotive_domain_test.jsonl --adapter checkpoints/automotive-adapter

2025-12-21 08:43:19.269034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766306599.448177   25591 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766306599.496780   25591 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766306599.873463   25591 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766306599.873505   25591 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766306599.873509   25591 computation_placer.cc:177] computation placer alr

In [9]:
!python scripts/evaluate_domain.py --adapter checkpoints/automotive-adapter

2025-12-18 09:11:10.124169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766049070.152316    5290 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766049070.162109    5290 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766049070.178369    5290 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766049070.178400    5290 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766049070.178404    5290 computation_placer.cc:177] computation placer alr

In [18]:
!python scripts/run_assistant.py --text "Error code P0420 detected on ECU"

2025-12-21 08:38:15.200749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766306295.242535   24314 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766306295.258332   24314 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766306295.290528   24314 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766306295.292859   24314 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766306295.292885   24314 computation_placer.cc:177] computation placer alr

In [6]:
!python scripts/run_assistant.py --text "Error code P0420 detected on ECU" --adapter checkpoints/automotive-adapter

2025-12-18 10:04:52.102593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766052292.274950   19089 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766052292.323080   19089 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766052292.679410   19089 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766052292.679459   19089 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766052292.679464   19089 computation_placer.cc:177] computation placer alr

In [20]:
!python scripts/run_assistant.py --text "Error code P0420 detected on ECU" --adapter checkpoints/automotive-adapter

2025-12-18 09:33:43.375702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766050423.430076   11033 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766050423.440312   11033 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766050423.475826   11033 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766050423.475879   11033 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766050423.475888   11033 computation_placer.cc:177] computation placer alr

In [13]:
!python scripts/experiments.py data/automotive_domain_test.jsonl --runs 5

2025-12-18 09:17:00.665600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766049420.687508    6774 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766049420.694544    6774 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766049420.709843    6774 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766049420.709869    6774 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766049420.709875    6774 computation_placer.cc:177] computation placer alr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
!pip install pyngrok

In [13]:
%ls

checkpoints/       notebooks/                           requirements-train.txt
config/            pipeline/                            requirements.txt
data/              QUICKSTART_EXPERIMENTS.md            role.md
DEPLOYMENT.md      README.md                            run.py
Dockerfile.edge    requirements-awq.txt                 scripts/
Dockerfile.orin    requirements-min.txt                 STRATEGIC_ALIGNMENT.md
IMPLEMENTATION.md  requirements-nlu.txt                 tests/
mlflow.db          requirements-rag.txt
models/            requirements-train-colab-freeze.txt


In [19]:
import subprocess
import time
import os
import requests
from pyngrok import ngrok

# 1. key needed for ngrok
ngrok.set_auth_token("36xtk7Ufu8VHyVjrd0qr5wMjK0g_47yU9mkyQ4foGARRAPC5s")


# 1. Define paths explicitly
project_dir = "/content/in-car-voice-assistant"
db_path = f"sqlite:///{project_dir}/mlflow.db"
log_file = f"{project_dir}/mlflow.log"

# 2. Kill any lingering processes safely
subprocess.run(["pkill", "-f", "gunicorn"])
subprocess.run(["pkill", "-f", "mlflow"])
time.sleep(2) # Give it a moment to die

# 3. Verify DB file exists
if not os.path.exists(f"{project_dir}/mlflow.db"):
    print(f"WARNING: {project_dir}/mlflow.db not found! Creating an empty one for testing...")
    # If the file is missing, MLflow might crash or create a temp one.
    # We let MLflow handle creation, but the path must be valid.

# 4. Start MLflow using Popen (Detached process)
print(f"Starting MLflow with DB: {db_path}...")
with open(log_file, "w") as out:
    process = subprocess.Popen(
        [
            "mlflow", "ui",
            "--backend-store-uri", db_path,
            "--host", "0.0.0.0",
            "--port", "5000"
        ],
        stdout=out,
        stderr=out,
        cwd=project_dir  # Run inside your project folder
    )
e
# 5. Wait and Check
print("Waiting for service to initialize...")
time.sleep(8)

# Check if process is still alive
if process.poll() is not None:
    print("FAILED: MLflow process died immediately.")
    print("--- Log Output ---")
    with open(log_file, "r") as f:
        print(f.read())
else:
    # Double check connection
    try:
        requests.get("http://127.0.0.1:5000")
        print("SUCCESS: MLflow is running locally.")

        # 6. Open Tunnel
        ngrok.kill()
        # ngrok.set_auth_token("YOUR_TOKEN") # Uncomment if needed
        tunnel = ngrok.connect(5000)
        print(f"DASHBOARD URL: {tunnel.public_url}")

    except Exception as e:
        print(f"Process is alive but not responding yet. Error: {e}")
        print("Check logs manually if the link below doesn't work.")

Starting MLflow with DB: sqlite:////content/in-car-voice-assistant/mlflow.db...
Waiting for service to initialize...
Process is alive but not responding yet. Error: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x784d69c38680>: Failed to establish a new connection: [Errno 111] Connection refused'))
Check logs manually if the link below doesn't work.


In [20]:
import time
import requests
from pyngrok import ngrok

print("Waiting for MLflow to accept connections...")

# 1. Loop and check if the server is ready (wait up to 60 seconds)
max_retries = 12
for i in range(max_retries):
    try:
        # Try to ping the local server
        response = requests.get("http://127.0.0.1:5000")
        if response.status_code == 200:
            print("\n✅ Success! MLflow is running.")
            break
    except requests.ConnectionError:
        print(f"   ... still loading (attempt {i+1}/{max_retries})")
        time.sleep(5)
else:
    # If the loop finishes without success
    print("\n❌ Timed out. Here is the log output to see what stuck:")
    print("="*40)
    !tail -n 20 /content/in-car-voice-assistant/mlflow.log
    print("="*40)
    raise RuntimeError("MLflow failed to start in time.")

# 2. Open the Tunnel (Only if success)
ngrok.kill()
# ngrok.set_auth_token("YOUR_TOKEN") # Uncomment if you need to set it again
tunnel = ngrok.connect(5000)
print(f"\n🔗 CLICK HERE: {tunnel.public_url}")

Waiting for MLflow to accept connections...
   ... still loading (attempt 1/12)

✅ Success! MLflow is running.

🔗 CLICK HERE: https://judie-apologetic-unhealthily.ngrok-free.dev


In [3]:
from google.colab import output

# This attempts to open port 5000 inside the output cell
output.serve_kernel_port_as_iframe(port=5000, height=600)

<IPython.core.display.Javascript object>

In [1]:
%cd in-car-voice-assistant

/content/in-car-voice-assistant


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = "microsoft/phi-2"
adapter = "checkpoints/automotive-adapter"

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base,
    trust_remote_code=True,
    # device_map="cpu" removed as it's not supported for 4-bit/8-bit models
)
model = PeftModel.from_pretrained(model, adapter)
model.eval() # Set model to evaluation mode

print("Loaded with adapters:", model.peft_config.keys())
print("Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# List the contents of the adapter directory to verify if files were saved correctly
!ls -F checkpoints/automotive-adapter

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

base = "microsoft/phi-2"
adapter = "checkpoints/automotive-adapter"

# 1. Define the Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)

# 2. Load model with config and device_map
model = AutoModelForCausalLM.from_pretrained(
    base,
    trust_remote_code=True,
    quantization_config=bnb_config, # Activates 4-bit loading
    device_map="auto"               # Offloads to GPU immediately
)

model = PeftModel.from_pretrained(model, adapter)
model.eval()

print("Loaded with adapters:", model.peft_config.keys())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded with adapters: dict_keys(['default'])


In [6]:
# Ensure pad token is set (often needed for Phi-2)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

def ask_the_mechanic(question, max_tokens=150):
    # IMPORTANT: Change this template to match exactly what you used during training!
    # If you trained on "### Instruction: {q} ### Response:", use that here.
    # Below is a generic safe default for Phi-2.
    prompt = f"Instruct: {question}\nOutput:"

    inputs = tok(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,      # Set False for deterministic answers
            temperature=0.7,     # Higher = more creative, Lower = more factual
            top_p=0.9,
            repetition_penalty=1.1, # Prevents it from repeating the same phrase
            pad_token_id=tok.pad_token_id
        )

    # Decode the output
    full_text = tok.decode(outputs[0], skip_special_tokens=True)

    # Strip the prompt to show only the new generated text
    try:
        answer = full_text.split("Output:")[-1].strip()
    except:
        answer = full_text

    return answer

# --- Sample Automotive Questions ---
test_questions = [
    "What are the symptoms of a failing alternator?",
    "How does a turbocharger increase engine power?",
    "Why does my car shake when I brake at high speeds?",
    "Open door",
    "check car health",
    "overview of car health",
    "open window",
]

print(f"Model is on device: {model.device}\n")

for q in test_questions:
    print(f"🔧 Q: {q}")
    print(f"📝 A: {ask_the_mechanic(q)}")
    print("-" * 60)

Model is on device: cuda:0

🔧 Q: What are the symptoms of a failing alternator?
📝 A: Symptoms of a failing alternator include dimming headlights, slow engine cranking, electrical system malfunctions (such as power windows or radio not working), and warning lights on the dashboard. It is important to address these issues promptly to prevent further damage to the vehicle's battery and other components.
------------------------------------------------------------
🔧 Q: How does a turbocharger increase engine power?
📝 A: A turbocharger increases engine power by forcing more air into the combustion chamber. It uses exhaust gases to spin a turbine, which in turn drives a compressor that compresses incoming air before it enters the engine cylinders. This compressed air-fuel mixture burns more efficiently, resulting in increased horsepower and torque.
------------------------------------------------------------
🔧 Q: Why does my car shake when I brake at high speeds?
📝 A: Brake noise and vibrati

## 3. AWQ Export

Export quantized model for edge deployment.

## 4. Comparison Table

| Configuration | Accuracy | Latency (ms) | Memory (GB) | Target Met |
|--------------|----------|--------------|-------------|------------|
| Baseline (rule) | 93.3% | 0.01 | <0.1 | ✓ Latency |
| + QLoRA | TBD | TBD | ~2.5 | TBD |
| + AWQ 4-bit | TBD | TBD | <2.0 | TBD |

In [ ]:
%cd in-car-voice-assistant/

/content/in-car-voice-assistant


In [ ]:
!python scripts/evaluate_intent.py data/automotive_domain_test.jsonl --classifier rule --rag off

Traceback (most recent call last):
  File "/content/in-car-voice-assistant/scripts/evaluate_intent.py", line 19, in <module>
    from pipeline.end_to_end import VoiceAssistantPipeline
ModuleNotFoundError: No module named 'pipeline'


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = "microsoft/phi-2"
adapter = "checkpoints/automotive-adapter"

tok = AutoTokenizer.from_pretrained(base, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base,
    trust_remote_code=True,
    # device_map="cpu" removed as it's not supported for 4-bit/8-bit models
)
model = PeftModel.from_pretrained(model, adapter)
model.eval() # Set model to evaluation mode

print("Loaded with adapters:", model.peft_config.keys())
print("Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install torch==2.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 749.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.4.0
    Uninstalling triton-3.4.0:
      Successfully uninstalled triton-3.4.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.27.3
    Uninstalling nvidia-nccl-cu12-2.27.3:
      Successfully uninstalled nvidia-nccl-cu12-2.27.3
  Attempting uninstall: torch
    Found existing installation: torch 2.8.0
    Uninstalling torch-2.8.0:
      Successfully uninstalled torch-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmcompressor 0.8.1 requires torch<=2.8.0,>=2.7.0, but you have torch 2.9.1 which is inco

## Archive

In [ ]:
# Recompile bitsandbytes from source
!git clone https://github.com/TimDettmers/bitsandbytes.git
%cd bitsandbytes
!CUDA_VERSION=126 python setup.py install
%cd ../


Cloning into 'bitsandbytes'...
remote: Enumerating objects: 11598, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 11598 (delta 98), reused 64 (delta 57), pack-reused 11443 (from 3)
Receiving objects: 100% (11598/11598), 3.78 MiB | 9.84 MiB/s, done.
Resolving deltas: 100% (7885/7885), done.
/content/in-car-voice-assistant/bitsandbytes
/usr/local/lib/python3.12/dist-packages/setuptools/_distutils/dist.py:261: UserWarning: Unknown distribution option: 'cmake_source_dir'
  warnings.warn(msg)
configuration error: `project.license` must be valid exactly by one definition (2 matches found):

    - keys:
        'file': {type: string}
      required: ['file']
    - keys:
        'text': {type: string}
      required: ['text']

DESCRIPTION:
    `Project license <https://peps.python.org/pep-0621/#license>`_.

GIVEN VALUE:
    "MIT"

OFFENDING RULE: 'oneOf'

DEFINITION:
    {
        "oneOf": [
            {
                "pr

In [ ]:
# Ensure peft is explicitly set to the compatible version in requirements-train.txt
!sed -i 's/^peft$/peft==0.7.1/g' requirements-train.txt
# Also update transformers, torch, bitsandbytes, and accelerate to known compatible versions
!sed -i 's/^transformers$/transformers==4.38.0/g' requirements-train.txt
!sed -i 's/^torch$/torch==2.1.0/g' requirements-train.txt
!sed -i 's/^bitsandbytes$/bitsandbytes==0.41.3/g' requirements-train.txt
!sed -i 's/^accelerate$/accelerate==0.26.1/g' requirements-train.txt
!cat requirements-train.txt

# Optional training/fine-tuning requirements
bitsandbytes
peft==0.7.1
transformers==4.38.0
torch
scikit-learn
pandas


In [ ]:
# Reinstall all requirements, ensuring compatible versions are used
!pip install -r requirements-train.txt --force-reinstall --upgrade

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.49.1.dev0-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached peft-0.7.1-py3-none-any.whl.metadata (25 kB)
  Using cached transformers-4.38.0-py3-none-any.whl.metadata (131 kB)
  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached scikit_learn-1.8.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
  Using cached tokenizer

In [ ]:
# Example of testing the fine-tuned model with a sample query

# Prepare an input query based on the training data format
query = "User: My check engine light is on. What should I do?"
input_ids = tok(query, return_tensors="pt").input_ids.cuda()

# Generate a response from the fine-tuned model
with torch.no_grad():
    output_tokens = model.generate(input_ids, max_new_tokens=50)

# Decode the output and print it
decoded_output = tok.decode(output_tokens[0], skip_special_tokens=True)
print("--- Fine-tuned Model Response ---")
print(decoded_output)
print("---------------------------------")

print("\nThis output demonstrates a basic inference step with your fine-tuned model.")
print("To perform a more formal evaluation and compare it against the baseline,")
print("we would integrate this into an evaluation script (like the one used for the baseline).")

[Errno 2] No such file or directory: 'in-car-voice-assistant/'
/content


In [ ]:
# Train QLoRA (requires GPU + requirements-train.txt)
!python scripts/train_qlora.py --model microsoft/phi-2 --epochs 3 --output checkpoints/automotive-adapter

2025-12-16 04:36:43.631197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765859803.652950   31160 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765859803.659935   31160 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765859803.677158   31160 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765859803.677182   31160 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765859803.677186   31160 computation_placer.cc:177] computation placer alr

In [ ]:
!pip install -r requirements-awq.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 826.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 798.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 139.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Reinstall remaining requirements-awq.txt dependencies (transformers, accelerate), which should keep torch==2.9.1
!pip install --force-reinstall -r requirements-awq.txt

  Using cached llmcompressor-0.8.1-py3-none-any.whl.metadata (11 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.3 MB/s eta 0:00:00
  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached nvidia_ml_py-13.580.82-py3-none-any.whl.metadata (9.6 kB)
  Using cached compressed_tensors-0.12.2-py3-none-any.whl.metadata (7.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manyl

In [ ]:
# Now install llmcompressor hoping it adapts to the existing torch==2.9.1, or gives a clear conflict
!pip install llmcompressor[transformers]

In [ ]:
!pip install llmcompressor

  Using cached llmcompressor-0.8.1-py3-none-any.whl.metadata (11 kB)
Using cached llmcompressor-0.8.1-py3-none-any.whl (273 kB)


In [ ]:
# Export AWQ (requires requirements-awq.txt)
!python scripts/export_awq.py --model microsoft/phi-2 --adapter checkpoints/automotive-adapter --output exports/automotive-awq-4bit

AWQ dependencies not available. Install requirements-awq.txt
pip install 'llm-compressor[transformers]'


In [ ]:
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
from transformers import AutoTokenizer, AutoModelForCausalLM

ModuleNotFoundError: Could not import module 'PreTrainedModel'. Are this object's requirements defined correctly?